# AI Face Mask Detection

## Pre-training

Import libraries and tools

In [ ]:
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from skorch import NeuralNetClassifier
# from skorch.helper import SliceDataset
from torch.utils.data import random_split
from sklearn.metrics import (make_scorer, accuracy_score, 
                        precision_score, recall_score, f1_score)
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import cross_validate

Transform methods

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.Resize(32),
    transforms.CenterCrop(32),
])

Hyperparameters

In [ ]:
num_epochs = 10
batch_size = 100
learning_rate = 1e-3
classes = ('cloth', 'n95', 'none', 'surgical')

Device to use

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## CNN architecture

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(4096, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        # conv layers
        x = self.conv_layer(x)
        # flatten
        x = x.view(x.size(0), -1)
        # fc layer
        x = self.fc_layer(x)
        return x

## Training

Import dataset

In [ ]:
dataset = torchvision.datasets.ImageFolder(root='./data/dataset/', transform=transform)

m = len(dataset)
train_dataset, test_dataset = random_split(dataset, [m - int(m / 4), int(m / 4)])

Train

In [ ]:
torch.manual_seed(0)
net = NeuralNetClassifier(
    CNN(),
    max_epochs=num_epochs,
    lr=learning_rate,
    batch_size=batch_size,
    optimizer=optim.Adam,
    criterion=nn.CrossEntropyLoss,
    device=device,
    iterator_train__shuffle=True,
)
y_train = np.array([np.int64(y) for x, y in iter(train_dataset)])
net.fit(train_dataset, y=y_train)

## Post-training

Train/test 75/25 split evaluation (**fast**)

In [ ]:
y_predict = net.predict(test_dataset)
y_test = np.array([y for x, y in iter(test_dataset)])
print('Accuracy for {} Dataset: {}%'.format('Test', round(accuracy_score(y_test, y_predict) * 100, 2)))
ConfusionMatrixDisplay.from_predictions(y_test, y_predict, display_labels=classes)
plt.title('Confusion Matrix for {} Dataset'.format('Test'))
plt.show()

K-fold cross-validation (**very slow**)

In [ ]:
k=10

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1_score': make_scorer(f1_score, average='weighted'),
}
y_train = np.array([np.int64(y) for x, y in iter(dataset)])
train_sliceable = SliceDataset(dataset)
scores = cross_validate(net, train_sliceable, y_train, cv=k, scoring=scoring)
print('\nScores:')
print('Accuracy: ', format(scores['test_accuracy']))
print('Precision: ', format(scores['test_precision']))
print('Recall: ', format(scores['test_recall']))
print('F1-measure: ', format(scores['test_f1_score']))